<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# # OR Unzipping to a specific directory
# !mkdir /content/CCAP
# !mkdir /content/CCAP/image_chips
!mkdir /content/CCAP/image_chips/Oahu

In [23]:
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/revised_trainingdata/Oahu.zip -d /content/CCAP/image_chips/Oahu/

Archive:  /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/revised_trainingdata/Oahu.zip
   creating: /content/CCAP/image_chips/Oahu/content/Oahu/
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/2005_Oahu_CCAP.tif  
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/2005_Oahu_LS.tif  
   creating: /content/CCAP/image_chips/Oahu/content/Oahu/Images/
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/Images/1_1.tif  
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/Images/1_10241.tif  
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/Images/1_1025.tif  
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/Images/1_10753.tif  
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/Images/1_11265.tif  
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/Images/1_11777.tif  
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/Images/1_12289.tif  
  inflating: /content/CCAP/image_chips/Oahu/content/Oahu/Images/1_12801.tif  
  inflating

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [25]:
df = pd.read_csv('/content/CCAP/image_chips/Oahu/content/Oahu/Oahu.csv')

In [26]:
df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
0,513,513,1,1,1,574284.807672,575513.611851,2.400140e+06,2.401369e+06,0,1.000000
1,1025,513,513,1,2,575513.576531,576742.367412,2.400140e+06,2.401369e+06,0,1.000000
2,1537,513,1025,1,3,576742.405848,577971.202058,2.400140e+06,2.401369e+06,0,1.000000
3,2049,513,1537,1,4,577971.200550,579200.034461,2.400140e+06,2.401369e+06,0,1.000000
4,2561,513,2049,1,5,579199.980574,580428.793004,2.400140e+06,2.401369e+06,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
8600,26113,21129,25601,20617,2265,635724.797437,636953.607199,2.350662e+06,2.351891e+06,0,0.779297
8601,26113,21129,25601,20617,2265,635724.797437,636953.607199,2.350662e+06,2.351891e+06,21,0.220703
8602,26625,21129,26113,20617,2266,636953.585761,638182.379739,2.350662e+06,2.351891e+06,0,1.000000
8603,27137,21129,26625,20617,2267,638182.368956,639411.189867,2.350662e+06,2.351891e+06,0,1.000000


In [27]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [33]:
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

<ipython-input-33-e8a0ba9a48fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"


In [34]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
38,12801,513,12289,1,25,603776.005882,605004.849318,2.400140e+06,2.401369e+06,15,0.003578,12289_1.tif
48,13313,513,12801,1,26,605004.808922,606233.600356,2.400140e+06,2.401369e+06,15,0.009815,12801_1.tif
59,13825,513,13313,1,27,606233.652296,607462.407113,2.400140e+06,2.401369e+06,15,0.006943,13313_1.tif
150,12801,1025,12289,513,79,603776.001254,605004.824022,2.398912e+06,2.400140e+06,14,0.001041,12289_513.tif
151,12801,1025,12289,513,79,603776.001254,605004.824022,2.398912e+06,2.400140e+06,15,0.012508,12289_513.tif
...,...,...,...,...,...,...,...,...,...,...,...,...
8451,20993,20993,20481,20481,2201,623436.851732,624665.616507,2.350988e+06,2.352217e+06,13,0.022396,20481_20481.tif
8452,20993,20993,20481,20481,2201,623436.851732,624665.616507,2.350988e+06,2.352217e+06,15,0.001633,20481_20481.tif
8492,25601,20993,25089,20481,2210,634496.011451,635724.800668,2.350988e+06,2.352217e+06,14,0.005688,25089_20481.tif
8558,20993,21129,20481,20617,2255,623436.824302,624665.654490,2.350662e+06,2.351891e+06,13,0.022396,20481_20617.tif
